In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Autoregressive Linear Regression on just Blood Glucose

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Load the data
file_path = '/content/drive/My Drive/Thesis/Subject5.xlsx'
cgm_data = pd.read_excel(file_path, sheet_name='CGM')

In [3]:
# Preprocess - Round to nearest 5 minutes
cgm_data['date'] = pd.to_datetime(cgm_data['date']).dt.round('5min')

# Filter out rows where mg/dl is > 400 or missing values
cgm_data = cgm_data[cgm_data['mg/dl'] <= 400].dropna(subset=['mg/dl'])

# Drop duplicates and sort
cgm_data = cgm_data.drop_duplicates(subset='date').reset_index(drop=True)
cgm_data = cgm_data.sort_values(by='date').reset_index(drop=True)

In [4]:
# Generate AR Features (using past 12 readings as input)
num_lags = 12 # Using the past 12 values
for lag in range(1, num_lags + 1):
    cgm_data[f'lag_{lag}'] = cgm_data['mg/dl'].shift(lag)

cgm_data = cgm_data.dropna().reset_index(drop=True)

In [5]:
# Split into train/test (80% train, 20% test by time order)
train_size = int(len(cgm_data) * 0.8)
train_data = cgm_data[:train_size]
test_data = cgm_data[train_size:]

In [6]:
# Define function to train and evaluate for different time steps
def train_ar_model(data, target_step):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Use np.sqrt on MSE

    # Display the coefficients
    coefficients = model.coef_
    intercept = model.intercept_

    print("Linear Regression Coefficients (lags):")
    for i, coef in enumerate(coefficients, start=1):
      print(f"Lag {i}: {coef:.4f}")
    print(f"Intercept: {intercept:.4f}")

    return model, rmse

In [7]:
# Train models for different time step targets and evaluate
time_steps = [3, 6, 9, 12]  # Corresponding to 15, 30, 45, 60 minutes
results = {}

for step in time_steps:
    model, rmse = train_ar_model(cgm_data, step)
    results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"Time step {step} (predicting {step*5} minutes ahead) - RMSE: {rmse:.4f}")

# Results dictionary now contains trained models and RMSE for each time target

Linear Regression Coefficients (lags):
Lag 1: 1.5462
Lag 2: -0.2692
Lag 3: -0.1585
Lag 4: -0.0720
Lag 5: -0.0106
Lag 6: -0.0330
Lag 7: -0.0286
Lag 8: -0.0141
Lag 9: -0.0117
Lag 10: -0.0025
Lag 11: 0.0044
Lag 12: -0.0196
Intercept: 12.5374
Time step 3 (predicting 15 minutes ahead) - RMSE: 16.4981
Linear Regression Coefficients (lags):
Lag 1: 1.6627
Lag 2: -0.3356
Lag 3: -0.2005
Lag 4: -0.0979
Lag 5: -0.0248
Lag 6: -0.0476
Lag 7: -0.0303
Lag 8: -0.0172
Lag 9: -0.0123
Lag 10: -0.0086
Lag 11: 0.0078
Lag 12: -0.0309
Intercept: 24.4628
Time step 6 (predicting 30 minutes ahead) - RMSE: 24.4678
Linear Regression Coefficients (lags):
Lag 1: 1.6676
Lag 2: -0.3637
Lag 3: -0.2199
Lag 4: -0.1017
Lag 5: -0.0295
Lag 6: -0.0502
Lag 7: -0.0386
Lag 8: -0.0177
Lag 9: -0.0124
Lag 10: -0.0024
Lag 11: 0.0126
Lag 12: -0.0488
Intercept: 37.0636
Time step 9 (predicting 45 minutes ahead) - RMSE: 31.1743
Linear Regression Coefficients (lags):
Lag 1: 1.6085
Lag 2: -0.3631
Lag 3: -0.2185
Lag 4: -0.1086
Lag 5: -0.0

ARX Model with Blood Glucose and Bolus Data

In [8]:
# Load Bolus data
bolus_data = pd.read_excel(file_path, sheet_name='Bolus')

# Preprocess Bolus data - Round to nearest 5 minutes and fill missing values with 0
bolus_data['date'] = pd.to_datetime(bolus_data['date']).dt.round('5min')
bolus_data = bolus_data.fillna(0)  # Set missing values to 0

# Merge Bolus data with CGM data on date
merged_data = pd.merge(cgm_data, bolus_data, on='date', how='left').fillna(0)  # Fill missing values after merge

# Generate lag features for Bolus variables
bolus_vars = ['normal', 'carbInput', 'insulinCarbRatio', 'bgInput', 'recommended.carb', 'recommended.net',
              'recommended.correction', 'insulinSensitivityFactor', 'targetBloodGlucose', 'insulinOnBoard']

for var in bolus_vars:
    for lag in range(1, num_lags + 1):
        merged_data[f'{var}_lag_{lag}'] = merged_data[var].shift(lag)

# Drop rows with NaNs introduced by shifting
merged_data = merged_data.dropna().reset_index(drop=True)

# Update train/test split based on merged data
train_data = merged_data[:train_size]
test_data = merged_data[train_size:]

<ipython-input-8-04aca2ebd074>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[f'{var}_lag_{lag}'] = merged_data[var].shift(lag)
<ipython-input-8-04aca2ebd074>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[f'{var}_lag_{lag}'] = merged_data[var].shift(lag)
<ipython-input-8-04aca2ebd074>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

In [9]:
# Modify train_ar_model function to include all lagged features
def train_arx_model(data, target_step):
    # Prepare inputs (X) - using lag features from both mg/dl and Bolus data
    lag_columns = [f'lag_{i}' for i in range(1, num_lags + 1)] + \
                  [f'{var}_lag_{i}' for var in bolus_vars for i in range(1, num_lags + 1)]
    X = data[lag_columns].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Extract coefficients and intercept
    coefficients = model.coef_
    intercept = model.intercept_

    # Display the coefficients
    print("Linear Regression Coefficients (ARX Model):")
    feature_names = merged_data.drop(columns=['date', 'mg/dl']).columns
    for feature, coef in zip(feature_names, coefficients):
      print(f"{feature}: {coef:.4f}")
    print(f"Intercept: {intercept:.4f}")

    return model, rmse

In [10]:
# Train ARX models for different time steps and evaluate
results_arx = {}
for step in time_steps:
    model, rmse = train_arx_model(merged_data, step)
    results_arx[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"Time step {step} (predicting {step*5} minutes ahead) - ARX RMSE: {rmse:.4f}")

Linear Regression Coefficients (ARX Model):
lag_1: 1.5389
lag_2: -0.2670
lag_3: -0.1575
lag_4: -0.0712
lag_5: -0.0102
lag_6: -0.0328
lag_7: -0.0285
lag_8: -0.0138
lag_9: -0.0112
lag_10: -0.0025
lag_11: 0.0048
lag_12: -0.0186
normal: 1.5214
carbInput: 1.4959
insulinCarbRatio: 1.4994
bgInput: 1.3765
recommended.carb: 1.0985
recommended.net: 0.9547
recommended.correction: 0.6918
insulinSensitivityFactor: 0.3898
targetBloodGlucose: 0.3202
insulinOnBoard: 0.2186
normal_lag_1: 0.4171
normal_lag_2: 0.5285
normal_lag_3: -0.0086
normal_lag_4: 0.0493
normal_lag_5: 0.0188
normal_lag_6: 0.0201
normal_lag_7: -0.2552
normal_lag_8: -0.4508
normal_lag_9: -0.4014
normal_lag_10: -0.5338
normal_lag_11: -0.2631
normal_lag_12: -0.3205
carbInput_lag_1: -0.3590
carbInput_lag_2: -0.2066
carbInput_lag_3: 0.1586
carbInput_lag_4: 0.6512
carbInput_lag_5: 0.4744
carbInput_lag_6: 0.5592
carbInput_lag_7: 0.5658
carbInput_lag_8: 0.5098
carbInput_lag_9: 0.4657
carbInput_lag_10: 0.0495
carbInput_lag_11: -0.2911
carbInp

Recursive AR Model

In [11]:
# Recursive forecasting function for the AR model
def recursive_forecast_ar_model(data, model, num_steps):
    # Initialize a list to store predictions
    predictions = []

    # Get the last 'num_lags' lag values from the data for initial input
    last_known_data = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].iloc[-1].values

    # Generate recursive predictions
    for _ in range(num_steps):
        # Reshape last known data to fit model input format
        next_input = last_known_data.reshape(1, -1)

        # Predict the next step
        next_pred = model.predict(next_input)[0]
        predictions.append(next_pred)

        # Update the input data by adding the new prediction and dropping the oldest value
        last_known_data = np.roll(last_known_data, -1)  # Shift values left
        last_known_data[-1] = next_pred  # Add new prediction as the latest lag

    return predictions

In [12]:
# Function to train the AR model on 1-step-ahead predictions
def train_single_step_ar_model(data):
    # Prepare inputs (X) and target (y) for single step
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-1).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions on the test set (1-step-ahead for evaluation)
    y_pred = model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return model, rmse

In [13]:
# Train the single-step AR model and evaluate
single_step_model, single_step_rmse = train_single_step_ar_model(cgm_data)
print(f"Single-step prediction model - RMSE: {single_step_rmse:.4f}")

# Perform recursive forecasting for different time steps
future_steps = [3, 6, 9, 12]  # 15, 30, 45, 60 minutes ahead
recursive_results = {}

for step in future_steps:
    # Recursive prediction
    predictions = recursive_forecast_ar_model(cgm_data, single_step_model, step)
    recursive_results[f'{step*5} min ahead'] = predictions
    print(f"Recursive prediction for {step*5} minutes ahead: {predictions}")

Single-step prediction model - RMSE: 9.9262
Recursive prediction for 15 minutes ahead: [337.5839078713668, 354.65831259882157, 379.013804851935]
Recursive prediction for 30 minutes ahead: [337.5839078713668, 354.65831259882157, 379.013804851935, 382.7365251466214, 350.7346734631458, 313.84221206502366]
Recursive prediction for 45 minutes ahead: [337.5839078713668, 354.65831259882157, 379.013804851935, 382.7365251466214, 350.7346734631458, 313.84221206502366, 305.24463075609754, 257.5197922044743, 262.1233244608659]
Recursive prediction for 60 minutes ahead: [337.5839078713668, 354.65831259882157, 379.013804851935, 382.7365251466214, 350.7346734631458, 313.84221206502366, 305.24463075609754, 257.5197922044743, 262.1233244608659, 308.24861977916646, 351.4782246485033, 355.6336226734991]


In [14]:
from sklearn.metrics import mean_squared_error

# Function to calculate RMSE for each recursive forecast
def calculate_recursive_rmse(data, model, num_steps):
    # Initialize list to store RMSE values
    rmse_values = {}

    # Iterate through each time step to calculate RMSE
    for step in num_steps:
        # Get the recursive predictions for the current step
        predictions = recursive_forecast_ar_model(data, model, step)

        # Get the actual values corresponding to this step
        actual_values = data['mg/dl'].shift(-step).dropna().values[:len(predictions)]

        # Calculate RMSE for this time step
        rmse = np.sqrt(mean_squared_error(actual_values, predictions))
        rmse_values[f'{step*5} min ahead'] = rmse
        print(f"RMSE for recursive forecast {step*5} minutes ahead: {rmse:.4f}")

    return rmse_values

# Calculate RMSE for recursive forecasts on the test set
recursive_rmse = calculate_recursive_rmse(cgm_data[train_size:], single_step_model, future_steps)


RMSE for recursive forecast 15 minutes ahead: 89.0505
RMSE for recursive forecast 30 minutes ahead: 71.9594
RMSE for recursive forecast 45 minutes ahead: 54.6455
RMSE for recursive forecast 60 minutes ahead: 46.4694


LASSO Regression

In [15]:
from sklearn.linear_model import Lasso

# Function to train and evaluate a Lasso regression model for a specified prediction step
def train_lasso_model(data, target_step, alpha=0.1, max_iter=5000):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the Lasso model with specified alpha (regularization strength)
    lasso_model = Lasso(alpha=alpha, max_iter=max_iter)
    lasso_model.fit(X_train, y_train)

    # Make predictions
    y_pred = lasso_model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return lasso_model, rmse

In [16]:
# Train and evaluate Lasso models for different time steps
lasso_results = {}
for step in time_steps:
    model, rmse = train_lasso_model(cgm_data, step, alpha=0.1, max_iter=5000)
    lasso_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"Lasso regression for {step*5} minutes ahead - RMSE: {rmse:.4f}")

Lasso regression for 15 minutes ahead - RMSE: 16.4987
Lasso regression for 30 minutes ahead - RMSE: 24.4686
Lasso regression for 45 minutes ahead - RMSE: 31.1751
Lasso regression for 60 minutes ahead - RMSE: 36.8410


Ridge Regression

In [17]:
from sklearn.linear_model import Ridge

# Function to train and evaluate a Ridge regression model for a specified prediction step
def train_ridge_model(data, target_step, alpha=1.0):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the Ridge model with specified alpha (regularization strength)
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)

    # Make predictions
    y_pred = ridge_model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return ridge_model, rmse

# Train and evaluate Ridge models for different time steps
ridge_results = {}
alpha_value = 1.0  # Adjust alpha as needed for regularization strength
for step in time_steps:
    model, rmse = train_ridge_model(cgm_data, step, alpha=alpha_value)
    ridge_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"Ridge regression for {step*5} minutes ahead - RMSE: {rmse:.4f}")


Ridge regression for 15 minutes ahead - RMSE: 16.4981
Ridge regression for 30 minutes ahead - RMSE: 24.4678
Ridge regression for 45 minutes ahead - RMSE: 31.1743
Ridge regression for 60 minutes ahead - RMSE: 36.8401


Elastic Net Regression

In [18]:
from sklearn.linear_model import ElasticNet

# Function to train and evaluate ElasticNet model
def train_elastic_net_model(data, target_step, alpha=1.0, l1_ratio=0.5):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the ElasticNet model
    elastic_net_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    elastic_net_model.fit(X_train, y_train)

    # Make predictions
    y_pred = elastic_net_model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return elastic_net_model, rmse

# Train and evaluate ElasticNet models for different time steps
elastic_net_results = {}
alpha_value = 1.0  # Regularization strength (larger values => more regularization)
l1_ratio_value = 0.5  # Mix ratio between Lasso (L1) and Ridge (L2) regularization
for step in time_steps:
    model, rmse = train_elastic_net_model(cgm_data, step, alpha=alpha_value, l1_ratio=l1_ratio_value)
    elastic_net_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"ElasticNet for {step*5} minutes ahead - RMSE: {rmse:.4f}")


ElasticNet for 15 minutes ahead - RMSE: 16.5064
ElasticNet for 30 minutes ahead - RMSE: 24.4763
ElasticNet for 45 minutes ahead - RMSE: 31.1826
ElasticNet for 60 minutes ahead - RMSE: 36.8497


Huber

In [19]:
from sklearn.linear_model import HuberRegressor

# Function to train and evaluate Huber regression model
def train_huber_model(data, target_step, epsilon=1.35):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the Huber regression model
    huber_model = HuberRegressor(epsilon=epsilon, max_iter=1000)
    huber_model.fit(X_train, y_train)

    # Make predictions
    y_pred = huber_model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return huber_model, rmse

# Train and evaluate Huber models for different time steps
huber_results = {}
epsilon_value = 1.35  # Huber loss function parameter (controls the threshold between quadratic and linear loss)
for step in time_steps:
    model, rmse = train_huber_model(cgm_data, step, epsilon=epsilon_value)
    huber_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"Huber for {step*5} minutes ahead - RMSE: {rmse:.4f}")


Huber for 15 minutes ahead - RMSE: 16.6951
Huber for 30 minutes ahead - RMSE: 24.5882
Huber for 45 minutes ahead - RMSE: 31.2031
Huber for 60 minutes ahead - RMSE: 36.7885


Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Function to train and evaluate Random Forest model
def train_rf_model(data, target_step, n_estimators=100, max_depth=None, random_state=42):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Train the Random Forest model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    rf_model.fit(X_train, y_train)

    # Make predictions
    y_pred = rf_model.predict(X_test)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return rf_model, rmse

# Train and evaluate Random Forest models for different time steps
rf_results = {}
for step in time_steps:
    model, rmse = train_rf_model(cgm_data, step, n_estimators=100, max_depth=10)
    rf_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"Random Forest for {step*5} minutes ahead - RMSE: {rmse:.4f}")


Random Forest for 15 minutes ahead - RMSE: 15.9138
Random Forest for 30 minutes ahead - RMSE: 23.6204
Random Forest for 45 minutes ahead - RMSE: 30.2032
Random Forest for 60 minutes ahead - RMSE: 35.7986


XGBoost

In [21]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Function to train and evaluate XGBoost model
def train_xgb_model(data, target_step, num_boost_round=100, learning_rate=0.1, max_depth=30, random_state=42):
    # Prepare inputs (X) and target (y)
    X = data[[f'lag_{i}' for i in range(1, num_lags + 1)]].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Set parameters for XGBoost
    params = {
        'objective': 'reg:squarederror',  # Regression task (squared error loss)
        'eval_metric': 'rmse',            # Metric to evaluate model
        'max_depth': max_depth,           # Depth of the trees
        'learning_rate': learning_rate,  # Step size shrinkage
        'random_state': random_state,    # For reproducibility
    }

    # Train the XGBoost model
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

    # Make predictions
    y_pred = model.predict(dtest)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return model, rmse

# Train and evaluate XGBoost models for different time steps
xgb_results = {}
for step in time_steps:
    model, rmse = train_xgb_model(cgm_data, step, num_boost_round=100, learning_rate=0.1, max_depth=10)
    xgb_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"XGBoost for {step*5} minutes ahead - RMSE: {rmse:.4f}")


XGBoost for 15 minutes ahead - RMSE: 15.9221
XGBoost for 30 minutes ahead - RMSE: 23.6233
XGBoost for 45 minutes ahead - RMSE: 30.2112
XGBoost for 60 minutes ahead - RMSE: 35.7944


In [22]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Function to train and evaluate XGBoost model with ARX features
def train_xgb_arx_model(data, target_step, num_boost_round=100, learning_rate=0.1, max_depth=10, random_state=42):
    # Prepare inputs (X) and target (y)
    # Use all lagged features (CGM + Bolus variables)
    feature_columns = [col for col in data.columns if col.startswith('lag_') or '_lag_' in col]
    X = data[feature_columns].values
    y = data['mg/dl'].shift(-target_step).dropna().values
    X = X[:len(y)]  # Align lengths

    # Split into train/test
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Set parameters for XGBoost
    params = {
        'objective': 'reg:squarederror',  # Regression task (squared error loss)
        'eval_metric': 'rmse',            # Metric to evaluate model
        'max_depth': max_depth,           # Depth of the trees
        'learning_rate': learning_rate,   # Step size shrinkage
        'random_state': random_state,     # For reproducibility
    }

    # Train the XGBoost model
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

    # Make predictions
    y_pred = model.predict(dtest)

    # Calculate performance metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return model, rmse

# Train and evaluate XGBoost models for different time steps
xgb_arx_results = {}
for step in time_steps:
    model, rmse = train_xgb_arx_model(merged_data, step, num_boost_round=100, learning_rate=0.1, max_depth=10)
    xgb_arx_results[f'{step*5} min ahead'] = {'Model': model, 'RMSE': rmse}
    print(f"XGBoost (ARX) for {step*5} minutes ahead - RMSE: {rmse:.4f}")

XGBoost (ARX) for 15 minutes ahead - RMSE: 15.8828
XGBoost (ARX) for 30 minutes ahead - RMSE: 23.4537
XGBoost (ARX) for 45 minutes ahead - RMSE: 30.0152
XGBoost (ARX) for 60 minutes ahead - RMSE: 35.7049
